# Install packages

First thing is to install pandas, numpy, scikit-learn, matplotlib, and seaborn. Then below you see where we import the packages in our notebook.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score




from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, ElasticNet, Lasso
from sklearn.metrics import classification_report, mean_squared_error, confusion_matrix, plot_confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler

# allow multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


# Import the data and start exploring it

In [ ]:

data = pd.read_csv('../Project Retain Alabama/Survey Data and Info/UABformatChange.csv')
alabamaData = pd.read_csv('../Project Retain Alabama/Survey Data and Info/alabamaData.csv')
alabamaData.describe()

Some commands you could start with : 
- data.describe()
- data.shape
- data.isnull().sum()
- data['column name'].value_counts()

# Transform the Data for Regression

In [ ]:
data = data.replace(np.NAN, 0)
alabamaData = alabamaData.replace(np.NAN, 0)
data = data.loc[:,~data.columns.duplicated()]
alabamaData = alabamaData.loc[:,~alabamaData.columns.duplicated()]

In [ ]:
# Use get dummies for one hot encoding 
# pd.get_dummies(data.University, prefix='University Code')
readData = pd.get_dummies(data, columns=['Race','StateHS','Major','Major Code', 'Major Code Revised'])
readAlabamaData = pd.get_dummies(alabamaData, columns=['Race','StateHS','Major','Major Code', 'Major Code Revised'])

readData
readAlabamaData.describe()
# data = pd.concat([data, rd], axis=1)
# data
# print(data)

In [ ]:
readData['StayAL'].value_counts()
alabamaData['StayAL'].value_counts()

In [ ]:
# A useful function for changing strings 
# for i in list(data.columns):
#     data[i] = data[i].apply(lambda x: str(x)).apply(lambda x: x.replace('|' , ','))
# data = data[~data.columns.duplicated()]
alabamaData.describe()

In [ ]:
# renaming columns
pd.options.display.max_columns = 300
pd.options.display.max_rows = 10
# readData
readAlabamaData
# data = data.rename(columns = {'_Virginia': 'Virginia'})
# data
# Regressiona

In [ ]:
# lstToKeep = [0,1]
readAlabamaData = readAlabamaData[readAlabamaData.StayAL != 3]

# Use multiple x values, especially ones that are dummies
# get rid of ALHS.1 'don't know'
# Split the dataset into two different ones, 
x = readAlabamaData['ALHS.1'].values.reshape(-1,1) 
print(x) #original is 8208 rows
print('mdddd')
y = readAlabamaData['StayAL'].values.reshape(-1,1)
print(y) #1206 rows 
# fix above this line. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train,y_train)
model.classes_
# print('Hoping for the best')
# model.predict_proba(y)
print('Hoping for the best x')
model.predict_proba(x)
print('The other bad boy')
y_predicted = model.predict(X_test)

# print('Holy moly')
# model.predict(y)
print('Lets get funky')

print('train')
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
# sc = StandardScaler()

# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(x, y)
plt.scatter(x,y)
plt.plot(y, regressor.predict(x),
         color = 'green')
# Y_pred = regressor.predict(X_train) # test the output by changing values




# confusion_matrix(y_test,y_pred)
# classification_report(y_test,y_pred)
# accuracy_score(y_test, y_pred)

# lin_reg = LinearRegression()
# lin_reg.fit(x, y)
# lin_reg.intercept_, lin_reg.coef_
# prayingToGod = lin_reg.predict(y)


# # 
# plt.scatter(x,y)
# plt.plot(x,prayingToGod,color='blue')
# plt.show()

In [ ]:
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)



# Model Validation

# 